In [1]:
#!/usr/bin/env python
from __future__ import print_function

from datetime import datetime
from collections import OrderedDict
import random
import os
import pandas as pd

import axioma
from axioma.assetset import AssetSet, ActionEntry
from axioma.account import Account
from axioma.workspace import DerbyProvider, Workspace
from axioma.workspace_element import ElementType
from axioma.costmodel import CostModel, CostStructure
from axioma.group import Group, Benchmark, Unit
from axioma.contentbuilder_group import ContentBuilderBenchmark, ContentBuilderGroup
from axioma.riskmodel import RiskModel
from axioma.strategy import Strategy, Objective, Target, Scope, MarketImpactType
from axioma.rebalancing import Rebalancing, RebalancingStatus
from axioma.workspace_element import ElementType
from axioma.metagroup import Metagroup, DynamicMetagroup
from axioma.analytics import Analytics
import axioma.workspace_io as handler




In [2]:
axioma.ENDPOINT="http://localhost:8085/axioma-websrv"

In [ ]:
from re import A, S
from numpy import maximum

# dates= Date column of AXWW51-calendar.att file, pd.read_csv()
dates = ["2024-10-02","2024-10-03"]

model = "WW51AxiomaSH"

# iterate through dates in dates list

############################## CREATE WORKSPACE
i=0
date = datetime.strptime(dates[i], '%Y-%m-%d').date()
next_period_date = datetime.strptime(dates[i+1], '%Y-%m-%d').date()
axioma_data_dir = r'C:/Users/jwbpa/Box/Altisma_Data/AxiomaDownloader-3.3.0-with-jre64/output/extractedFiles/database/${yyyy}/${mm}/'
db_provider = DerbyProvider(axioma_data_dir,
                            risk_models=model,
                            include_composites=True,
                            next_period_date=next_period_date,
                            returns_type="Gross Return")

ws = Workspace(f"AltismaWorkspace_{dates[i]}", date, data_provider=db_provider)



############################## LOAD DATA

ac_df = pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/AlphaCapture_Net_MA_{dates[i]}.csv", index_col="Name")
ac_account = Account(workspace = ws, 
        identity = "AC_Input", 
        holdings=ac_df.to_dict()["Shares"], 
        asset_map="Ticker Map")

prev_day_optimized_ac_df=pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/"OutPut"_{dates[i-1]}.csv", index_col="Name")
prev_day_optimized_ac_account = Account(workspace = ws, 
        identity = "prev_day_optimized_ac_Input", 
        holdings=prev_day_optimized_ac_df.to_dict()["Shares"], 
        asset_map="Ticker Map")

## Content Builder Attributes
# Content Builder for 60-Day MDV
Inv_60_Day_MDV = ContentBuilderGroup(workspace=ws, identity = "Inv_60_Day_MDV", expression = "(1/'60-Day MDV')")

# Content Builder for Account Currency
Account_Currency = ContentBuilderGroup(workspace=ws, identity = "Account_Currency", expression = "portfolioAsCurrency('AC_Input')*1")


############################## DEFINE STRATEGY
strategy = Strategy(workspace = ws, 
                    identity = "Strategy", 
                    allow_shorting=True, 
                    allow_crossover=False, 
                    enable_constraint_hierarchy=True)
strategy.set_local_universe(local_universe=list(ac_account.get_holdings.keys()) + ["CSH_USD__"])
############################## DEFINE OBJECTIVE TERMS
ar_term = axioma.strategy.create_risk_term(strategy = strategy, 
                                            identity = "activeRisk", 
                                            benchmark=Account_Currency, 
                                            risk_model=model,
                                            asset_set="MASTER")
tc_term = axioma.strategy.create_market_impact_term(strategy = strategy, 
                                                    identity = "marketImpact",
                                                    buy_impact_group=Inv_60_Day_MDV,
                                                    sell_impact_group=Inv_60_Day_MDV,
                                                    market_impact_type=MarketImpactType.Quadratic
                                                    )
############################## DEFINE OBJECTIVE FUNCTION
terms = OrderedDict()
terms[ar_term] = 1.0
terms[tc_term] = 1.0
obj_fx = Objective(strategy = strategy, 
                    identity = "Objective", 
                    terms=terms, 
                    target=Target.Minimize, 
                    active=True)
############################## DEFINE CONSTRAINTS
# constraint 1
leverage_limit=axioma.strategy.create_limit_absolute_leverage_constraint(strategy = strategy,
                                                                         identity="leverageLimit",
                                                                         minimum=100,
                                                                         maximum=200,
                                                                         unit=Unit.Percent,
                                                                         scope=Scope.Aggregate
                                                                         )

leverage_limit.add_selection(element_type=ElementType.AssetSet, element="MASTER")

# constraint 2
rm = ws.get_risk_model(model)
factor_risk_limit_except_industry=axioma.strategy.create_limit_factor_relative_marginal_contribution_to_risk_constraint(strategy = strategy,
                                                                                                                 identity = "factorRiskLimitExceptIndustry",
                                                                                                                 maximum=0.2, 
                                                                                                                 risk_model=model, 
                                                                                                                 scope=Scope.Aggregate,
                                                                                                                 risk_model_group_factors=rm.get_factor_names(),
                                                                                                                 include_specific_risk=True
                                                                                                                 )
style_mgp = ws.get_metagroup(f"{model}.Style")
for group in style_mgp.get_groups():
    factor_risk_limit_except_industry.add_selection(element_type=ElementType.Group, element=group)

country_mgp = ws.get_metagroup(f"{model}.Country")
for group in country_mgp.get_groups():
    factor_risk_limit_except_industry.add_selection(element_type=ElementType.Group, element=group)

currency_mgp = ws.get_metagroup(f"{model}.Currency")
for group in currency_mgp.get_groups():
    factor_risk_limit_except_industry.add_selection(element_type=ElementType.Group, element=group)

market_mgp = ws.get_metagroup(f"{model}.Market")
for group in market_mgp.get_groups():
    factor_risk_limit_except_industry.add_selection(element_type=ElementType.Group, element=group)

local_mgp = ws.get_metagroup(f"{model}.Local")
for group in local_mgp.get_groups():
    factor_risk_limit_except_industry.add_selection(element_type=ElementType.Group, element=group)


# Attribute=Group in API, Classification = Metagroup in API, Factors=Group in API, Sets in GUI is AssetSet in API.

# constraint 3
factor_risk_itemized_limit = axioma.strategy.create_limit_factor_relative_marginal_contribution_to_risk_constraint(strategy = strategy,
                                                                                                            identity = "factorRiskItemizedLimit",
                                                                                                            maximum=0.05, 
                                                                                                            risk_model=model, 
                                                                                                            scope=Scope.Selection,
                                                                                                            risk_model_group_factors=rm.get_factor_names(),
                                                                                                            include_specific_risk=True
                                                                                                            )
# multi-selection: Market_Sensitivity, Medium-Term Momentum, Residual Volatility
factor_risk_itemized_limit.add_selection(element_type=ElementType.Group, element=f"{model}.Market Sensitivity")
factor_risk_itemized_limit.add_selection(element_type=ElementType.Group, element=f"{model}.Medium-Term Momentum")
factor_risk_itemized_limit.add_selection(element_type=ElementType.Group, element=f"{model}.Residual Volatility")



# constraint 4
stock_GMV_conserve=axioma.strategy.create_limit_absolute_holding_constraint(strategy = strategy,
                                                                           identity = "stockGMVConserve",
                                                                           maximum=5.0,
                                                                           benchmark=Account_Currency,
                                                                           scope=Scope.Asset,
                                                                           unit=Unit.Percent
                                                                           )

# Selection: NON-CASH ASSETS
stock_GMV_conserve.add_selection(element_type=ElementType.AssetSet, element="NON-CASH ASSETS")



# constraint 5
dollar_neutral=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                               identity = "dollarNeutral",
                                                               minimum=-5,
                                                               maximum=5,
                                                               scope=Scope.Aggregate,
                                                               unit=Unit.Percent
                                                               )

# Selection: NON-CASH ASSETS
dollar_neutral.add_selection(element_type=ElementType.AssetSet, element="NON-CASH ASSETS")                



# constraint 6
industry_factor_limit = axioma.strategy.create_limit_factor_relative_marginal_contribution_to_risk_constraint(
    strategy=strategy,
    identity="industryFactorLimit",
    maximum=0.1,
    risk_model=model,
    scope=Scope.Aggregate,
    risk_model_group_factors=rm.get_factor_names(),
    include_specific_risk=True
)

# all industry factors
industry_mgp = ws.get_metagroup(f"{model}.Industry")
for group in industry_mgp.get_groups():
    industry_factor_limit.add_selection(element_type=ElementType.Group, element=group)




# constraint 7
stock_idio_risk_conserve=axioma.strategy.create_limit_total_risk_constraint(strategy = strategy,
                                                                     identity = "stockIdioRiskConserve",
                                                                     factor_weight=0.0,
                                                                     maximum=2,
                                                                     benchmark=Account_Currency,
                                                                     risk_model=model,
                                                                     scope=Scope.Selection,
                                                                     specific_weight=1.0,
                                                                     unit=Unit.Percent
                                                                     )

# Selection: MASTER
stock_idio_risk_conserve.add_selection(element_type=ElementType.AssetSet, element="MASTER")



# constraint 8
budget=axioma.strategy.create_budget_constraint(strategy = strategy,
                                                identity = "budget",
                                                use_budget_value=True
                                                )




############################## SET CONSTRAINT HIERARCHY CONSTRAINTS
#strategy.set_constraint_hierarchy({"06. Match Style Factor Exposures of K-I" : 1, "05. Match Industry Exposures of K-I" : 2})
#strategy.set_constraint_hierarchy({"06. Match Style Factor Exposures of K-I" : 1})

############################## CREATE REBALANCING
rebal = Rebalancing(workspace = ws, 
                    identity="AC_Rebalancing",
                    account=prev_day_optimized_ac_account, 
                    strategy=strategy
                    )

#rebal.set_reference_size(1000000)
#rebal.set_budget_size(1000000)
#rebal.set_benchmark_size(1000000)

rebal.set_rebalancing_defaults(round_lot_size=1,
                               use_cash_for_roundlot=True,
                               risk_model=model)



############################## SOLVE REBALANCING
sol = rebal.solve()
print(sol.get_status())
if sol.get_status()==RebalancingStatus.SolutionFound or sol.get_status()==RebalancingStatus.RelaxedSolutionFound:
    fh = sol.get_final_holdings(asset_map = "Ticker Map")





############################## CALCULATE ANALYTICS - THESE ARE EXAMPLES OF SOME AVAILABLE ANALYTICS
# create Analytics object. This will be used to calculate analytics
analyzer = Analytics(workspace = ws, price_group="Price", asset_map="Ticker Map")

# compute active holdings of final holdings
# ah = analyzer.compute_active_holdings(holdings=fh, benchmark=kmi, reference_value=rebal.get_reference_size())

# calculate active exposures. Returns exposure in dollar term for each factor -> dic to dataframe-> dollar value to percentage in decimal
fh_exposures = pd.DataFrame.from_dict({"Exposure":analyzer.compute_factor_exposures(risk_model=model,holdings=fh)})/rebal.get_reference_size()

# compute tracking error. Compute active holdings and then calculate total risk
#delta_holdings = analyzer.compute_active_holdings(holdings=fh, benchmark=alpha_capture, reference_value=alpha_capture.get_composition_sum())
#delta_risk = analyzer.compute_total_risk(risk_model=model,holdings=delta_holdings)/rebal.get_reference_size()*100

########################## Find Idio Return and Risk of final solution next day.
# Factor return
factor_return_dollar = analyzer.compute_factor_return_contributions(risk_model=model, 
                                                          metagroup = f"{model}.Period Returns",
                                                          holdings=fh)

# Total return
total_return_dollar =analyzer.compute_expected_return(alphas="Period Return", holdings=fh)

# Idio return
idio_return_dollar = total_return_dollar - factor_return_dollar

# Convert dollar to %
idio_return_percent = idio_return_dollar / rebal.get_reference_size()*100

# Factor risk
factor_risk_dollar = analyzer.compute_factors_risk(risk_model=model, holdings=fh)

# Total risk
total_risk_dollar = analyzer.compute_total_risk(risk_model=model, holdings=fh)

# Idio risk
idio_risk_dollar = analyzer.compute_specific_risk(risk_model=model, holdings=fh)

# Convert dollar to %
idio_risk_percent = idio_risk_dollar / rebal.get_reference_size()*100


# write workspace to .wsp file and release license token
ws.write(os.getcwd(),file_name=f"workspace_on_{dates[i]}.wsp",save_reference=False)
ws.destroy()



SyntaxError: expected argument value expression (2718769150.py, line 136)